In [ ]:
from google.colab import drive
drive.mount('/content/drive') # library to read file in google drive

Mounted at /content/drive


In [ ]:
import numpy as np   # library to help organize the data format
import matplotlib.pyplot as plt   # library to make plot
import random
import csv
import pandas as pd 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Dropout,BatchNormalization,Input,concatenate
from keras import Model
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from matplotlib import pyplot
from sklearn import tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

In [ ]:
#load data form google drive, if you are using local development environment,
#change the urlto your local data addrerss
import pandas as pd

df_ob = pd.read_csv('/content/drive/MyDrive/sql/obsolete.csv')
print("Completed loading Obsolete data")

df_no = pd.read_csv('/content/drive/MyDrive/sql/non_obsolete.csv')
print("Completed loading No_obsolete data")



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Completed loading Obsolete data
Completed loading No_obsolete data


In [ ]:
print("answer data types\n {}".format(df_ob.dtypes))
print(df_ob.shape)

answer data types
 c_id              int64
a_id              int64
text             object
c_date           object
u_id            float64
a_id_1          float64
a_date           object
score           float64
c_count         float64
u_id_1          float64
p_id            float64
p_id_1          float64
p_date           object
a_count         float64
tags             object
u_id_2          float64
a_reputation    float64
u_id_3          float64
c_reputation    float64
dtype: object
(195184, 19)


In [ ]:
df_ob=df_ob.dropna(how='all')
df_no=df_no.dropna(how='all')

In [ ]:
df_ob['target']=1
df_no['target']=0

In [ ]:
df = pd.concat([df_ob,df_no])
df = df.reset_index()
df

,index,c_id,a_id,text,c_date,u_id,a_id_1,a_date,score,c_count,...,p_id,p_id_1,p_date,a_count,tags,u_id_2,a_reputation,u_id_3,c_reputation,target
0,0,80917928,46924726,"Sorry, but answering these sort of questions w...",2017-10-28 01:10:24,6375498.0,46924726.0,2017-10-25 05:39:26,2.0,17.0,...,46852221.0,46852221.0,2017-10-20 15:17:33,2.0,<java><sockets><web><communication>,6375498.0,1910.0,6375498.0,1910.0,1
1,1,40579377,25859560,As an alternative to using `tempfile.Temporary...,2014-09-18 21:33:40,4014959.0,25859560.0,2014-09-16 01:54:33,2.0,16.0,...,25856065.0,25856065.0,2014-09-15 20:05:16,1.0,<python><pygame>,4014959.0,48928.0,4014959.0,48928.0,1
2,2,14038037,10791062,Regarding the layer `display` call (assuming i...,2012-05-29 00:42:18,792677.0,10791062.0,2012-05-28 22:30:12,1.0,12.0,...,10791025.0,10791025.0,2012-05-28 22:26:15,3.0,<ios><uiview><core-animation><calayer><drawrect>,792677.0,8808.0,792677.0,8808.0,1
3,3,97090264,55175764,"So in that case, read the second and third cod...",2019-03-15 05:05:32,1553090.0,55175764.0,2019-03-15 04:54:04,2.0,16.0,...,55175594.0,55175594.0,2019-03-15 04:30:32,2.0,<c++>,1553090.0,50611.0,1553090.0,50611.0,1
4,4,37005028,23949868,"Yes You can use any String, that´s not importa...",2014-06-03 07:45:01,1538986.0,23949868.0,2014-05-30 08:07:59,5.0,15.0,...,23949296.0,23949296.0,2014-05-30 07:30:24,1.0,<android>,1538986.0,9305.0,1538986.0,9305.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395179,199995,22066388,15568139,"I added ""java -Xmx16m class"" in .bashrc file. ...",2013-03-22 10:41:42,1782239.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1782239.0,27.0,0
395180,199996,5362614,4831017,"@Luigi: Technically, ""configuration"" type file...",2011-01-28 19:48:40,13792.0,4831017.0,2011-01-28 17:09:34,2.0,7.0,...,4830547.0,4830547.0,2011-01-28 16:20:57,8.0,<java><maven><layout><resources>,298455.0,46317.0,13792.0,101799.0,0
395181,199997,53817,166052,"@finnw : I agree, but then, Java developers sh...",2008-10-03 10:58:59,14089.0,166052.0,2008-10-03 09:13:00,1.0,9.0,...,166033.0,166033.0,2008-10-03 09:04:08,4.0,<java><c++><terminology><semantics>,18715.0,6180.0,14089.0,73646.0,0
395182,199998,84226481,48448033,"Maybe try $rs and $rr outside the """" ?",2018-02-05 01:27:14,9092533.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9092533.0,180.0,0


In [ ]:
df = df[['score','c_date','a_date','c_count','p_date','a_count','tags','a_reputation','target']]
df

,score,c_date,a_date,c_count,p_date,a_count,tags,a_reputation,target
0,2.0,2017-10-28 01:10:24,2017-10-25 05:39:26,17.0,2017-10-20 15:17:33,2.0,<java><sockets><web><communication>,1910.0,1
1,2.0,2014-09-18 21:33:40,2014-09-16 01:54:33,16.0,2014-09-15 20:05:16,1.0,<python><pygame>,48928.0,1
2,1.0,2012-05-29 00:42:18,2012-05-28 22:30:12,12.0,2012-05-28 22:26:15,3.0,<ios><uiview><core-animation><calayer><drawrect>,8808.0,1
3,2.0,2019-03-15 05:05:32,2019-03-15 04:54:04,16.0,2019-03-15 04:30:32,2.0,<c++>,50611.0,1
4,5.0,2014-06-03 07:45:01,2014-05-30 08:07:59,15.0,2014-05-30 07:30:24,1.0,<android>,9305.0,1
...,...,...,...,...,...,...,...,...,...
395179,NaN,2013-03-22 10:41:42,NaN,NaN,NaN,NaN,NaN,NaN,0
395180,2.0,2011-01-28 19:48:40,2011-01-28 17:09:34,7.0,2011-01-28 16:20:57,8.0,<java><maven><layout><resources>,46317.0,0
395181,1.0,2008-10-03 10:58:59,2008-10-03 09:13:00,9.0,2008-10-03 09:04:08,4.0,<java><c++><terminology><semantics>,6180.0,0
395182,NaN,2018-02-05 01:27:14,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
df = df.dropna()
df

,score,c_date,a_date,c_count,p_date,a_count,tags,a_reputation,target
0,2.0,2017-10-28 01:10:24,2017-10-25 05:39:26,17.0,2017-10-20 15:17:33,2.0,<java><sockets><web><communication>,1910.0,1
1,2.0,2014-09-18 21:33:40,2014-09-16 01:54:33,16.0,2014-09-15 20:05:16,1.0,<python><pygame>,48928.0,1
2,1.0,2012-05-29 00:42:18,2012-05-28 22:30:12,12.0,2012-05-28 22:26:15,3.0,<ios><uiview><core-animation><calayer><drawrect>,8808.0,1
3,2.0,2019-03-15 05:05:32,2019-03-15 04:54:04,16.0,2019-03-15 04:30:32,2.0,<c++>,50611.0,1
4,5.0,2014-06-03 07:45:01,2014-05-30 08:07:59,15.0,2014-05-30 07:30:24,1.0,<android>,9305.0,1
...,...,...,...,...,...,...,...,...,...
395174,2.0,2020-12-05 03:39:58,2020-12-05 02:45:25,1.0,2020-12-05 01:58:05,2.0,<javascript><regex>,36.0,0
395175,0.0,2016-10-06 05:46:35,2016-10-05 17:50:58,6.0,2016-10-05 17:42:22,1.0,<python><python-2.7>,3821.0,0
395180,2.0,2011-01-28 19:48:40,2011-01-28 17:09:34,7.0,2011-01-28 16:20:57,8.0,<java><maven><layout><resources>,46317.0,0
395181,1.0,2008-10-03 10:58:59,2008-10-03 09:13:00,9.0,2008-10-03 09:04:08,4.0,<java><c++><terminology><semantics>,6180.0,0


In [ ]:
df[df['target']==1].count()

score           82779
c_date          82779
a_date          82779
c_count         82779
p_date          82779
a_count         82779
tags            82779
a_reputation    82779
target          82779
dtype: int64

In [ ]:
#change date type all to datetime
df['c_date'] = pd.to_datetime(df['c_date'],format='%Y-%m-%dT%H:%M:%S.%f')
df['a_date'] = pd.to_datetime(df['a_date'],format='%Y-%m-%dT%H:%M:%S.%f')
df['p_date'] = pd.to_datetime(df['p_date'],format='%Y-%m-%dT%H:%M:%S.%f')

print("----------------DATA TYPES -------------\n{}".format(df.dtypes))

----------------DATA TYPES -------------
score                  float64
c_date          datetime64[ns]
a_date          datetime64[ns]
c_count                float64
p_date          datetime64[ns]
a_count                float64
tags                    object
a_reputation           float64
target                   int64
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
df['a_time'] = (df['a_date']-df['p_date']).astype('timedelta64[m]')
df['c_time'] = (df['c_date']-df['a_date']).astype('timedelta64[m]')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,score,c_date,a_date,c_count,p_date,a_count,tags,a_reputation,target,a_time,c_time
0,2.0,2017-10-28 01:10:24,2017-10-25 05:39:26,17.0,2017-10-20 15:17:33,2.0,<java><sockets><web><communication>,1910.0,1,6621.0,4050.0
1,2.0,2014-09-18 21:33:40,2014-09-16 01:54:33,16.0,2014-09-15 20:05:16,1.0,<python><pygame>,48928.0,1,349.0,4059.0
2,1.0,2012-05-29 00:42:18,2012-05-28 22:30:12,12.0,2012-05-28 22:26:15,3.0,<ios><uiview><core-animation><calayer><drawrect>,8808.0,1,3.0,132.0
3,2.0,2019-03-15 05:05:32,2019-03-15 04:54:04,16.0,2019-03-15 04:30:32,2.0,<c++>,50611.0,1,23.0,11.0
4,5.0,2014-06-03 07:45:01,2014-05-30 08:07:59,15.0,2014-05-30 07:30:24,1.0,<android>,9305.0,1,37.0,5737.0
...,...,...,...,...,...,...,...,...,...,...,...
395174,2.0,2020-12-05 03:39:58,2020-12-05 02:45:25,1.0,2020-12-05 01:58:05,2.0,<javascript><regex>,36.0,0,47.0,54.0
395175,0.0,2016-10-06 05:46:35,2016-10-05 17:50:58,6.0,2016-10-05 17:42:22,1.0,<python><python-2.7>,3821.0,0,8.0,715.0
395180,2.0,2011-01-28 19:48:40,2011-01-28 17:09:34,7.0,2011-01-28 16:20:57,8.0,<java><maven><layout><resources>,46317.0,0,48.0,159.0
395181,1.0,2008-10-03 10:58:59,2008-10-03 09:13:00,9.0,2008-10-03 09:04:08,4.0,<java><c++><terminology><semantics>,6180.0,0,8.0,105.0


In [ ]:
df['tags']= df['tags'].str[1:-1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['tags']= df['tags'].str.replace('><',' ')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,score,c_date,a_date,c_count,p_date,a_count,tags,a_reputation,target,a_time,c_time
0,2.0,2017-10-28 01:10:24,2017-10-25 05:39:26,17.0,2017-10-20 15:17:33,2.0,java sockets web communication,1910.0,1,6621.0,4050.0
1,2.0,2014-09-18 21:33:40,2014-09-16 01:54:33,16.0,2014-09-15 20:05:16,1.0,python pygame,48928.0,1,349.0,4059.0
2,1.0,2012-05-29 00:42:18,2012-05-28 22:30:12,12.0,2012-05-28 22:26:15,3.0,ios uiview core-animation calayer drawrect,8808.0,1,3.0,132.0
3,2.0,2019-03-15 05:05:32,2019-03-15 04:54:04,16.0,2019-03-15 04:30:32,2.0,c++,50611.0,1,23.0,11.0
4,5.0,2014-06-03 07:45:01,2014-05-30 08:07:59,15.0,2014-05-30 07:30:24,1.0,android,9305.0,1,37.0,5737.0
...,...,...,...,...,...,...,...,...,...,...,...
395174,2.0,2020-12-05 03:39:58,2020-12-05 02:45:25,1.0,2020-12-05 01:58:05,2.0,javascript regex,36.0,0,47.0,54.0
395175,0.0,2016-10-06 05:46:35,2016-10-05 17:50:58,6.0,2016-10-05 17:42:22,1.0,python python-2.7,3821.0,0,8.0,715.0
395180,2.0,2011-01-28 19:48:40,2011-01-28 17:09:34,7.0,2011-01-28 16:20:57,8.0,java maven layout resources,46317.0,0,48.0,159.0
395181,1.0,2008-10-03 10:58:59,2008-10-03 09:13:00,9.0,2008-10-03 09:04:08,4.0,java c++ terminology semantics,6180.0,0,8.0,105.0


In [ ]:
df = shuffle(df)
df

,score,c_date,a_date,c_count,p_date,a_count,tags,a_reputation,target,a_time,c_time
202795,0.0,2009-12-03 05:01:14,2009-12-03 03:43:15,4.0,2009-12-03 03:13:10,2.0,objective-c cocoa,43193.0,0,30.0,77.0
121433,16.0,2015-11-04 20:10:02,2014-09-17 17:21:30,1.0,2014-09-17 17:21:30,2.0,julia,3232.0,1,0.0,594888.0
324749,1.0,2014-12-04 17:16:52,2014-12-03 05:14:49,6.0,2014-12-02 20:41:41,1.0,jsf primefaces refactoring code-reuse,624.0,0,513.0,2162.0
152935,15.0,2016-05-26 17:21:20,2016-05-26 16:50:03,3.0,2016-05-26 16:42:49,1.0,pointers rust,247147.0,1,7.0,31.0
274015,0.0,2012-11-14 13:40:50,2012-11-14 12:07:42,2.0,2012-11-14 11:36:14,2.0,wpf data-binding progress,4280.0,0,31.0,93.0
...,...,...,...,...,...,...,...,...,...,...,...
184096,0.0,2016-01-02 09:23:58,2016-01-02 07:56:32,7.0,2016-01-02 07:44:39,2.0,php mysql,6300.0,1,11.0,87.0
114019,17.0,2017-07-12 04:58:59,2014-01-09 06:22:41,1.0,2014-01-08 21:59:12,5.0,angularjs twitter-bootstrap bower angular-ui-b...,13149.0,1,503.0,1843116.0
292558,0.0,2013-10-13 13:17:14,2013-10-13 05:49:20,5.0,2013-10-12 17:59:33,2.0,session cakephp sessionid,189.0,0,709.0,447.0
337692,2.0,2015-07-16 14:59:28,2015-07-14 12:21:40,2.0,2015-07-14 10:37:23,1.0,xmpp publish-subscribe smack,21804.0,0,104.0,3037.0


In [ ]:
df_x1 = df[['score','c_count','a_count','a_reputation','a_time','c_time']]
df_x2 = df['tags']
df_y = df['target']

In [ ]:
scaler = MinMaxScaler()
# transform data
df_x1 = scaler.fit_transform(df_x1)
df_x1

array([[1.58903115e-03, 3.44827586e-02, 1.93423598e-03, 3.52581711e-02,
        1.15638587e-01, 2.48183414e-05],
       [2.31544538e-03, 8.62068966e-03, 1.93423598e-03, 2.63750581e-03,
        1.15634441e-01, 9.22887087e-02],
       [1.63443203e-03, 5.17241379e-02, 0.00000000e+00, 5.08562710e-04,
        1.15705332e-01, 3.48232352e-04],
       ...,
       [1.58903115e-03, 4.31034483e-02, 1.93423598e-03, 1.53466757e-04,
        1.15732417e-01, 8.22107558e-05],
       [1.67983292e-03, 1.72413793e-02, 0.00000000e+00, 1.77980622e-02,
        1.15648813e-01, 4.83957657e-04],
       [1.63443203e-03, 1.72413793e-02, 1.93423598e-03, 6.73784368e-03,
        1.15775947e-01, 6.24181286e-03]])

In [ ]:
df_tag = df_x2.str.split(' ')
li = list(df_tag)
dic = {}
for i in li:
  for j in i:
    if j in dic:
      dic[j] = dic[j]+1
    else:
      dic[j] = 1
dic
myDict = {key:val for key, val in dic.items() if val> 100}
len(myDict.keys())

660

In [ ]:
myDict.keys()

dict_keys(['objective-c', 'cocoa', 'jsf', 'primefaces', 'refactoring', 'pointers', 'rust', 'wpf', 'data-binding', 'java', 'string', 'c#', 'asp.net-core', 'python', 'python-3.x', 'list', 'duplicates', 'delphi', 'generics', 'c++', 'interface', 'android', 'maven-2', 'git', 'iphone', 'xcode', 'join', 'php', 'css', 'html', 'image', 'visual-studio-2015', 'symfony', 'serialization', '.net', 'vb.net', 'ios', 'swift', 'webview', 'javascript', 'forms', 'jquery', 'datetime', 'algorithm', 'asp.net', 'web-applications', 'asp.net-mvc', 'reference', '3d', 'geolocation', 'g++', 'reactjs', 'react-router', 'facebook', 'twitter-bootstrap', 'node.js', 'excel', 'google-chrome', 'internet-explorer', 'windows', 'clojure', 'numpy', 'mongodb', 'math', 'ruby-on-rails', 'ruby', 'rubygems', 'nhibernate', 'python-2.7', 'selenium', 'meteor', 'wordpress', 'azure', 'sql', 'mysql', 'angular', 'winapi', 'hyperlink', 'matlab', 'swing', 'netbeans', 'rest', 'post', 'playframework', 'security', 'qt', 'character-encoding', 

In [ ]:
li1 = []
for i in li:
  string = ''
  for j in i:
    if j in myDict.keys():
      string = string + j +' '
  li1.append(string) 
df_x2 = pd.DataFrame(li1)
df_x2

,0
0,objective-c cocoa
1,
2,jsf primefaces refactoring
3,pointers rust
4,wpf data-binding
...,...
177190,php mysql
177191,angularjs twitter-bootstrap
177192,session cakephp
177193,


In [ ]:
df_x2['tags'] = df_x2
df_x2['tags'] = df_x2['tags'].str.split(' ')

In [ ]:
df_x2 = df_x2['tags']

In [ ]:
def identity_tokenizer(text):
    return text
#transfer into tf-idf format
tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)    
X2 = tfidf.fit_transform(df_x2)

In [ ]:
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(df_x1, X2, df_y, test_size=0.3, random_state = 30)

# **Cart decision tree**

In [ ]:
dt = tree.DecisionTreeClassifier()

tree_para = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}

grid_search = GridSearchCV(estimator = dt, param_grid = tree_para, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X1_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30,
                                       40, 50, 70, 90, 120, 150]},
             verbose=2)

In [ ]:
grid_search.best_params_

{'criterion': 'entropy', 'max_depth': 5}

In [ ]:
best_grid = grid_search.best_estimator_
y_pred=best_grid.predict(X1_test)

In [ ]:
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.6530032543877801


In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.85916884, 0.41748277])

# **Random forest**

In [ ]:
rf = RandomForestClassifier(bootstrap= True,max_depth= 90,max_features= 2,min_samples_leaf= 5,min_samples_split= 8,n_estimators= 1000)
# Train the model on training data
rf.fit(X1_train, y_train)

RandomForestClassifier(max_depth=90, max_features=2, min_samples_leaf=5,
                       min_samples_split=8, n_estimators=1000)

In [ ]:
y_pred=rf.predict(X1_test)


In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6532101807784194


In [ ]:
matrix = confusion_matrix(y_test, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.80848347, 0.47782718])

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 300, 1000]
}

In [ ]:
rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(X1_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
y_pred=best_grid.predict(X1_test)


In [ ]:
print("Accuracy:",accuracy_score(y_test, y_pred))

In [ ]:
matrix = confusion_matrix(y_test, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

# **ANN**

In [ ]:
# define two sets of inputs
inputA = Input(shape=(6,))

# the first branch operates on the first input
x = Dense(4, activation="relu")(inputA)
x = Dense(2, activation="relu")(x)
z = Dense(1, activation="sigmoid")(x)
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=inputA, outputs=z)

In [ ]:
opt = Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(X1_train,y_train,validation_data=(X1_test, y_test), batch_size=200, epochs=200)

Epoch 1/200
621/621 [==============================] - 4s 4ms/step - loss: 0.6538 - accuracy: 0.6306 - val_loss: 0.6553 - val_accuracy: 0.6252
Epoch 2/200
621/621 [==============================] - 1s 2ms/step - loss: 0.6523 - accuracy: 0.6324 - val_loss: 0.6541 - val_accuracy: 0.6271
Epoch 3/200
621/621 [==============================] - 1s 2ms/step - loss: 0.6511 - accuracy: 0.6336 - val_loss: 0.6532 - val_accuracy: 0.6292
Epoch 4/200
621/621 [==============================] - 2s 3ms/step - loss: 0.6502 - accuracy: 0.6347 - val_loss: 0.6525 - val_accuracy: 0.6303
Epoch 5/200
621/621 [==============================] - 1s 2ms/step - loss: 0.6494 - accuracy: 0.6357 - val_loss: 0.6520 - val_accuracy: 0.6313
Epoch 6/200
621/621 [==============================] - 1s 2ms/step - loss: 0.6488 - accuracy: 0.6366 - val_loss: 0.6516 - val_accuracy: 0.6319
Epoch 7/200
621/621 [==============================] - 1s 2ms/step - loss: 0.6483 - accuracy: 0.6375 - val_loss: 0.6512 - val_accuracy: 0.6325

In [ ]:
y_pred = model.predict(X1_test)
y_pred 

array([[0.38888055],
       [0.74571896],
       [0.35807592],
       ...,
       [0.3301487 ],
       [0.36705232],
       [0.8200282 ]], dtype=float32)

In [ ]:
y_p = np.where(y_pred > 0.5, 1, 0)
y_p

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
matrix = confusion_matrix(y_test, y_p)
matrix.diagonal()/matrix.sum(axis=1)

array([0.87402468, 0.38653207])

# **Multi input**

In [ ]:
X2_train = X2_train.toarray()
X2_test = X2_test.toarray()

In [ ]:
# define two sets of inputs
inputA = Input(shape=(6,))
inputB = Input(shape=(661,))
# the first branch operates on the first input
x = Dense(4, activation="relu")(inputA)
x = Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = Dense(256, activation="relu")(inputB)
y = Dense(64, activation="relu")(y)
y = Dense(4, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = concatenate([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="sigmoid")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)

In [ ]:
opt = Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit([X1_train,X2_train],y_train,validation_data=([X1_test,X2_test], y_test), batch_size=200, epochs=200)

Epoch 1/200
621/621 [==============================] - 11s 15ms/step - loss: 0.6819 - accuracy: 0.5431 - val_loss: 0.6726 - val_accuracy: 0.5847
Epoch 2/200
621/621 [==============================] - 5s 9ms/step - loss: 0.6684 - accuracy: 0.5969 - val_loss: 0.6667 - val_accuracy: 0.5986
Epoch 3/200
621/621 [==============================] - 5s 8ms/step - loss: 0.6630 - accuracy: 0.6097 - val_loss: 0.6642 - val_accuracy: 0.6046
Epoch 4/200
621/621 [==============================] - 5s 8ms/step - loss: 0.6594 - accuracy: 0.6153 - val_loss: 0.6622 - val_accuracy: 0.6088
Epoch 5/200
621/621 [==============================] - 5s 8ms/step - loss: 0.6561 - accuracy: 0.6212 - val_loss: 0.6604 - val_accuracy: 0.6118
Epoch 6/200
621/621 [==============================] - 5s 9ms/step - loss: 0.6529 - accuracy: 0.6257 - val_loss: 0.6592 - val_accuracy: 0.6117
Epoch 7/200
621/621 [==============================] - 5s 8ms/step - loss: 0.6495 - accuracy: 0.6328 - val_loss: 0.6570 - val_accuracy: 0.61

In [ ]:
y_pred = model.predict([X1_test,X2_test])
y_pred 

array([[0.1543026 ],
       [0.85239124],
       [0.4834972 ],
       ...,
       [0.05658236],
       [0.32267034],
       [0.85239124]], dtype=float32)

In [ ]:
y_p = np.where(y_pred > 0.5, 1, 0)
y_p

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
matrix = confusion_matrix(y_test, y_p)
matrix.diagonal()/matrix.sum(axis=1)

array([0.74758831, 0.517566  ])

# **KNN**

In [ ]:
parameters = {"n_neighbors": range(1, 50)}
grid_search = GridSearchCV(KNeighborsRegressor(), parameters)
grid_search.fit(X1_train, y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 50)})

In [ ]:
grid_search.best_params_

{'n_neighbors': 49}

In [ ]:
best_grid = grid_search.best_estimator_
y_pred=best_grid.predict(X1_test)

In [ ]:
y_pred.shape[0]

53159

In [ ]:
y_p = np.where(y_pred > 0.5, 1, 0)
y_pred

array([0.32653061, 0.79591837, 0.30612245, ..., 0.75510204, 0.36734694,
       0.36734694])

In [ ]:
y_p

array([0, 1, 0, ..., 1, 0, 0])

In [ ]:
y_test

298725    0
141445    1
185954    1
196493    0
195127    1
         ..
134759    1
315453    0
179390    1
116136    1
253616    0
Name: target, Length: 53159, dtype: int64

In [ ]:
print("Accuracy:",accuracy_score(y_test, y_p))

Accuracy: 0.6417727948230779


In [ ]:
matrix = confusion_matrix(y_test, y_p)
matrix.diagonal()/matrix.sum(axis=1)

array([0.81814013, 0.44351751])